## TODO
1. add features
2. change targets
3. model
4. loss_fn
5. tune the parameters

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## packages and settings

In [ ]:
import numpy as np 
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import os
import copy
import sys

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
from sklearn.metrics import log_loss, roc_auc_score

from sklearn.model_selection import GroupKFold
from tqdm import tqdm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

if not os.path.exists("results"):
    os.mkdir("results")

TRAINING = True
read_path = '/kaggle/input/jane-street-market-prediction/train.csv'
model_path = "/kaggle/input/skeleton-with-pytorch/results/123/best_model"
save_path = os.path.join("results", str(seed))

device = torch.device("cuda:0")
if not os.path.exists(save_path):
    os.mkdir(save_path)
    
# train = pd.read_csv(read_path)

In [ ]:
TRAINING = False

## preprocess the data

In [ ]:
train = pd.read_csv(read_path)
train = train.query('date > 85').reset_index(drop = True) 
features = [c for c in train.columns if 'feature' in c]

# preprocess the features
f_mean = train[features].mean()
train = train.loc[train.weight > 0].reset_index(drop = True)
train[features] = train[features].fillna(f_mean)

train = train.astype("float32")
train['action'] = (train['resp'] > 0).astype('int')
train['action1'] = (train['resp_1'] > 0).astype('int')
train['action2'] = (train['resp_2'] > 0).astype('int')
train['action3'] = (train['resp_3'] > 0).astype('int')
train['action4'] = (train['resp_4'] > 0).astype('int')

targets = ['action']
# train[targets] = (train[targets]>0).astype('int')


def add_features(df, features):
    new_features = copy.deepcopy(features)
    
    # todo
    df["cross_1_2"] = df["feature_1"] / (df["feature_2"] + 1e-5)
    df["cross_41_42_43"] = df["feature_41"] + df["feature_42"] + df["feature_43"]
    new_features.extend(["cross_1_2", "cross_41_42_43"])

    return df, new_features

train, train_features = add_features(train, features)


# to do: update the mean online
# f_mean = f_mean.values
# np.save(os.path.join(save_path, 'f_mean.npy'), f_mean)

In [ ]:
# reduce memory
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [ ]:
train = reduce_memory_usage(train)

## the dataset and model(resnet)

In [ ]:
class MyDataset:
    def __init__(self, df, features, targets):
        self.features = df[features].values
        
        # preprocess the labels
        # self.labels = (df[targets] > 0).astype('int').values
        self.labels = df[targets].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feat_ = torch.tensor(self.features[idx], dtype=torch.float)
        label_ = torch.tensor(self.labels[idx], dtype=torch.float)
        
        return feat_, label_

    
# def MyDataset2(df,features,targets,time_step):
#     row = df.shape[0]
#     row = row - row%(time_step)
#     df = df.iloc[:row,:]
#     df1 = list(df[features].values.reshape(1,-1,time_step,len(features)))
#     df1.append(list(df[targets].values.reshape(-1,1)))
#     df1 = list(zip(*df1))
#     return df1
    

In [ ]:
from torch.utils.data import Dataset
class MyDataset2(Dataset):
    def __init__(self, df, features, targets, time_step):
        super().__init__()
        self.features = df[features].values
        # preprocess the labels
        # self.labels = (df[targets] > 0).astype('int').values
        self.labels = df[targets].values

    def __len__(self):
        return len(self.labels)-time_step+1

    def __getitem__(self, idx):
        feat_ = torch.tensor(self.features[idx:idx+time_step], dtype=torch.float)
        feat_ = feat_.unsqueeze(dim=0)
        label_ = torch.tensor(self.labels[idx+time_step-1], dtype=torch.float)
        
        return feat_, label_


## for test

In [ ]:
tmp = np.array([[1,1,1,1,1,1],[2,2,2,2,2,2],[3,3,3,3,3,3],[4,4,4,4,4,4]])
tmp.reshape(1,-1,2,6)

In [ ]:
tmp_data = MyDataset2(train.iloc[:100,:], train_features, targets, 10)

### end test

In [ ]:
class Model(nn.Module):
    def __init__(self, features, targets, time_step):
        super(Model, self).__init__()
        #self.batch_norm0 = nn.BatchNorm1d(len(features))
        self.batch_norm0 = nn.BatchNorm2d(1)
        self.dropout0 = nn.Dropout(0.10143786981358652)

        hidden_size = 256
        self.dense1 = nn.Linear(len(features), 384)
        #self.batch_norm1 = nn.BatchNorm1d(384)
        self.batch_norm1 = nn.BatchNorm2d(1)
        self.dropout1 = nn.Dropout(0.19720339053599725)

        self.dense2 = nn.Linear(384, 896)
        #self.batch_norm2 = nn.BatchNorm1d(896)
        self.batch_norm2 = nn.BatchNorm2d(1)
        self.dropout2 = nn.Dropout(0.2703017847244654)

        self.dense3 = nn.Linear(896, 896)
        #self.batch_norm3 = nn.BatchNorm1d(896)
        self.batch_norm3 = nn.BatchNorm2d(1)
        self.dropout3 = nn.Dropout(0.23148340929571917)

        self.dense4 = nn.Linear(896, 394)
        #self.batch_norm4 = nn.BatchNorm1d(394)
        self.batch_norm4 = nn.BatchNorm2d(1)
        self.dropout4 = nn.Dropout(0.2357768967777311)

        #self.dense5 = nn.Linear(394, len(targets))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()
        
        self.rnn = nn.LSTM(input_size=394,hidden_size=64,num_layers=2,dropout=0.2,batch_first=True)
        self.hidden_out = nn.Linear(64,len(targets))
        self.h_s = None
        self.h_c = None

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x = self.dense1(x)
        x = self.batch_norm1(x)
        x = x * torch.sigmoid(x)
        x = self.dropout1(x)

        x = self.dense2(x)
        x = self.batch_norm2(x)
        x = x * torch.sigmoid(x)
        x = self.dropout2(x)
        
        x = self.dense3(x)
        x = self.batch_norm3(x)
        x = x * torch.sigmoid(x)
        x = self.dropout3(x)
        
        x = self.dense4(x)
        x = self.batch_norm4(x)
        x = x * torch.sigmoid(x)
        x = self.dropout4(x)
        
        #print(x.shape)
        x = x.squeeze(dim=1) #try
        #print(x.shape)
        
        x,(h_s,h_c) = self.rnn(x)
        x = self.hidden_out(x)

        #x = self.dense5(x)

        return x
    
    
    
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

    
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            self.save_checkpoint(epoch_score, model, model_path)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved. Saving model!')
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

## Train

### utility

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0

    for feature, label in dataloader:
        feature = feature.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        outputs = model(feature)
        
        loss = loss_fn(outputs[:,-1,:], label) #只取最后一个时间点的输出
        
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

        final_loss += loss.item()

    final_loss /= len(dataloader)
    return final_loss
        
    
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    labels = []

    for feature, label in dataloader:
        feature = feature.to(device)
        # label = label.to(device)
        with torch.no_grad():
            outputs = model(feature)
            preds.append(outputs.sigmoid().cpu().numpy())
            labels.append(label.cpu().numpy())
    
    preds = np.concatenate(preds, axis=0)
    labels = np.concatenate(labels, axis=0)

    return preds, labels
    
    
def utility_score(date, weight, resp, action):    
    values = weight * resp * action
    to_bincount = {}

    for d, v in zip(date, values):
        to_bincount.setdefault(d, []).append(v)

    Pi = []
    for val in to_bincount.values():
        Pi.append(np.sum(val))
    Pi = np.array(Pi)
    count_i = len(np.unique(date))
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    
    return u
    
    
def loss_mse(preds, targets):
    
    return ((preds-targets)**2).mean()


def loss_ce(preds, targets, weight=None):
    
    return F.binary_cross_entropy_with_logits(preds, targets, weight)

In [ ]:
# PurgedGroupTimeSeriesSplit——根据时序划分数据集
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
if TRAINING:
    batch_size = 4096
    label_smoothing = 1e-2
    learning_rate = 1e-3
    time_step = 10
    
    import time
    start_time = time.time()
    oof = np.zeros(len(train['action']))
    cv = PurgedGroupTimeSeriesSplit(n_splits=2, group_gap=20,max_test_group_size=80)
    splits = list(cv.split(train[train_features].values, groups=train['date'].values))
    
    for fold, (tr, te) in enumerate(splits):
        #train_set = MyDataset(train.loc[tr], train_features, targets)
        train_set = MyDataset2(train.loc[tr], train_features, targets, time_step)
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
        #valid_set = MyDataset(train.loc[te], train_features, targets)
        valid_set = MyDataset2(train.loc[te], train_features, targets, time_step)
        valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=0)

        torch.cuda.empty_cache()
        device = torch.device("cuda:0")
        model = Model(train_features, targets, time_step)
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        loss_fn = SmoothBCEwLogits(smoothing=label_smoothing)

        ckp_path = f'JSModel_test_{fold}.pth'

        es = EarlyStopping(patience=3, mode="max")

        for epoch in range(10):
            train_loss = train_fn(model, optimizer, None, loss_fn, train_loader, device)
            
            train_pred, true_train_labels = inference_fn(model, train_loader, device)
            train_pred = train_pred[:,-1,:]
            true_train_labels = true_train_labels[:]
            #auc_score_tr = roc_auc_score((train.loc[tr]['resp'] > 0).astype('int').values.reshape(-1, 1), train_pred)
            auc_score_tr = roc_auc_score(true_train_labels, train_pred)
            #logloss_score = log_loss((train.loc[te]['resp'] > 0).astype('int').values.reshape(-1, 1), valid_pred)
            train_pred = np.where(train_pred >= 0.5, 1, 0).astype(int)
            u_score_tr = utility_score(date=train.loc[tr].date.values.reshape(-1)[time_step-1:],
                                    weight=train.loc[tr].weight.values.reshape(-1)[time_step-1:],
                                    resp=train.loc[tr].resp.values.reshape(-1)[time_step-1:],
                                    action=train_pred.reshape(-1))
            
            valid_pred, true_valid_labels = inference_fn(model, valid_loader, device)
            valid_pred = valid_pred[:,-1,:]
            #auc_score_te = roc_auc_score((train.loc[te]['resp'] > 0).astype('int').values.reshape(-1, 1), valid_pred)
            auc_score_te = roc_auc_score(true_valid_labels, valid_pred)
            #logloss_score = log_loss((train.loc[te]['resp'] > 0).astype('int').values.reshape(-1, 1), valid_pred)
            valid_pred = np.where(valid_pred >= 0.5, 1, 0).astype(int)
            u_score_te = utility_score(date=train.loc[te].date.values.reshape(-1)[time_step-1:],
                                    weight=train.loc[te].weight.values.reshape(-1)[time_step-1:],
                                    resp=train.loc[te].resp.values.reshape(-1)[time_step-1:],
                                    action=valid_pred.reshape(-1))

            print(f"FOLD{fold} EPOCH:{epoch:3}, train_loss:{train_loss:.5f}, u_score_tr:{u_score_tr:.5f}, auc_tr:{auc_score_tr:.5f},"
                  f"u_score_te:{u_score_te:.5f}, auc_te:{auc_score_te:.5f}, "
                  f"time: {(time.time() - start_time) / 60:.2f}min")

            es(auc_score_te, model, model_path=ckp_path)
            if es.early_stop:
                print("Early stopping")
                break
    #     break # only train 1 model for fast, you can remove it to train 5 folds

In [ ]:
from torchsummary import summary

summary(model, input_size=(1, time_step, 132))

In [ ]:
models = []
for i in [0, 2, 4]: # for fast inference, you can change 1-->5 to get higher score
    torch.cuda.empty_cache()
    device = torch.device("cuda:0")
    model = Model(train_features, targets)
    model.to(device)
    model.eval()
    
    ckp_path = f'/kaggle/input/models/JSModel_{i}.pth'
    model.load_state_dict(torch.load(ckp_path))
    models.append(model)

In [ ]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

th = 0.5

for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        
        if np.isnan(x_tt.sum()):
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean.values.reshape(1, -1)

        feature_inp = pd.DataFrame(x_tt)
        feature_inp.columns = features
        feature_inp, _ = add_features(feature_inp,features)
        feature_inp = torch.tensor(feature_inp.values, dtype=torch.float).to(device)
        
        pred = np.zeros((1, len(targets)))
        for model in models: 
            pred += model(feature_inp).sigmoid().detach().cpu().numpy()
        pred /= len(models)
        
        pred = pred.mean(axis=1).item()
        pred_df.action = int(pred >= th)
        
    else:
        pred_df.action = 0
        
    env.predict(pred_df)